In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import cv2
import random
import shutil

from shutil import copyfile

import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, MaxPooling2D, Flatten, Dropout, Conv2D, GlobalAveragePooling2D, GlobalMaxPool2D
from tensorflow.keras.optimizers import Adam

In [15]:
base = ''

DATAPATH = os.path.join('datasets/Testset/')
MASK_PATH = os.path.join(DATAPATH,'well_ported_mask/')
NO_MASK_PATH = os.path.join(DATAPATH,'no_mask/')
WRONG_MASK_PATH = os.path.join(DATAPATH,'wrong_ported_mask/')
WEIGHTS_PATH = os.path.join(base,'weights/')

initial_weights = os.path.join(WEIGHTS_PATH,'3classes_FineTunning_mobTrainable.h5')
trainable = True

In [16]:
def view(pth):
    images = list()
    for img in random.sample(os.listdir(pth),9):
        images.append(img)
    i = 0
    fig,ax = plt.subplots(nrows=3, ncols=3, figsize=(30,20))
    for row in range(3):
        for col in range(3):
            ax[row,col].imshow(cv2.imread(os.path.join(pth,images[i])))
            i+=1

In [17]:
NumberMaskData = os.listdir(MASK_PATH)
print(len(NumberMaskData))

407


In [18]:
NumberNOMaskData = os.listdir(NO_MASK_PATH)
print(len(NumberNOMaskData))

179


In [19]:
NumberWrongMaskData = os.listdir(WRONG_MASK_PATH)
print(len(NumberWrongMaskData))

481


In [20]:
fig = go.Figure(
    data=[go.Pie(labels=['WITHMASK','WITHOUTMASK','WRONGMASK'], 
        values=[NumberMaskData , NumberNOMaskData , NumberWrongMaskData ])
    ])
fig.show()

In [21]:
testGen = ImageDataGenerator(
    rescale= 1.0/255.0,
)

In [22]:
test = testGen.flow_from_directory(
    DATAPATH, 
    target_size=(128, 128),
    classes=['no_mask','well_ported_mask', 'wrong_ported_mask'],
    class_mode='categorical', 
    batch_size=1, 
    shuffle=False,
)


Found 1065 images belonging to 3 classes.


In [23]:
mob = MobileNetV2(
    input_shape = (128,128,3),
    include_top = False,
    weights = 'imagenet',
)
mob.trainable = trainable


In [24]:
model = Sequential()
model.add(mob)
model.add(GlobalMaxPool2D())
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(3,activation='softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_128 (Functi (None, 4, 4, 1280)        2257984   
_________________________________________________________________
global_max_pooling2d_1 (Glob (None, 1280)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                81984     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 195       
Total params: 2,340,163
Trainable params: 82,179
Non-trainable params: 2,257,984
_________________________________________________________________


In [25]:
model.compile(loss='categorical_crossentropy',metrics=['accuracy'])
model.load_weights( initial_weights )

In [26]:
model.evaluate(test)

1065/1065 [==============================] - 10s 10ms/step - loss: 0.8813 - accuracy: 0.5568


[0.8812596797943115, 0.5568075180053711]